In [9]:
%run partialOrder.ipynb

R = {(1, 4),(1, 5),(2, 4),(3, 6)}
S = {(4, 9),(5, 8),(6, 7)}
R∘S = {(1, 8),(1, 9),(2, 9),(3, 7)}

is binary relation? R = True
is binary relation? S = True

is homogeneous relation? R = False
is homogeneous relation? S = False

is identity relation? R = False
is identity relation? S = False

S[A] = FiniteSet(8, 9)
S-1[A] = FiniteSet(5, 6)

dom S = FiniteSet(4, 5, 6)
rng S = FiniteSet(7, 8, 9)

R == S: False
R != S: True
R < S: False
R <= S: False
R > S: False
R >= S: False

R | S = {(1, 4),(1, 5),(2, 4),(3, 6),(4, 9),(5, 8),(6, 7)}
R & S = {}
R - S = {(1, 4),(1, 5),(2, 4),(3, 6)}
R ^ S = {(1, 4),(1, 5),(2, 4),(3, 6),(4, 9),(5, 8),(6, 7)}

R complement = {(1, 6),(2, 5),(2, 6),(3, 4),(3, 5)}
R∘S converse = {(7, 3),(8, 1),(9, 1),(9, 2)}

'1' is in relation with: [4, 5] in R
[4, 5]
[1, 2]
True
2
<class '__main__.Number'>
True
False
False
False
True
R = {(1, 1),(2, 1),(2, 2),(3, 1),(3, 2),(3, 3)}
[3]
[1]
3
1
True
True
True
2
20
2
60
1
60
3
60
----------
True
False
[1]
1
[60]
60
True
False
[

In [2]:
import logging
from lark import Lark, logger, Tree, Token

parser = Lark('''
                RELATION: "P".."Z"
                SET: "A".."O"
                ?logical_operator: set_product
                     | logical_operator "=="  set_product  -> equal
                     | logical_operator "!="  set_product  -> not_equal
                     | logical_operator ">"   set_product  -> greater_than
                     | logical_operator "<"   set_product  -> lower_than
                     | logical_operator ">="  set_product  -> greater_equal
                     | logical_operator "<="  set_product  -> lower_equal
                     | logical_operator "and" set_product -> conjunction
                     | logical_operator "or"  set_product  -> disjunction
                     | logical_operator "->"  set_product  -> implication
                 ?set_product: item
                     | set_product "&" item   -> intersection
                     | set_product "|" item   -> union
                     | set_product "-" item   -> difference
                     | set_product "^" item   -> symmetric_difference
                     | set_product "*" item   -> multiplication
                 ?item: SET
                     | RELATION
                     | "("logical_operator")"
                 %import common.WS
                 %ignore WS
         ''', start='logical_operator')

In [3]:
A = FiniteSet(1,2,3,4)
B = FiniteSet(3,4,5,6)

tree = parser.parse("(((A & B) <= (A | B)) or ((A*B) > (A*B))) -> (((A != B) and (B == B)) -> (A >= (B - (A ^ B))))")        

In [4]:
def constant(data):
        try:
            return globals()[data]
        except:
            raise Exception(f"Global variable - '{data}' doesn't exist!")

def interpret(node):
    if isinstance(node, Tree):
        left, right = node.children
        if node.data == 'equal':
            return interpret(left) == interpret(right)
        elif node.data == 'not_equal':
            return interpret(left) != interpret(right)
        elif node.data == 'greater_than':
            return interpret(left) > interpret(right)
        elif node.data == 'lower_than':
            return interpret(left) < interpret(right)
        elif node.data == 'greater_equal':
            return interpret(left) >= interpret(right)
        elif node.data == 'lower_equal':
            return interpret(left) <= interpret(right)
        elif node.data == 'conjunction':
            return interpret(left) and interpret(right)
        elif node.data == 'disjunction':
            return interpret(left) or interpret(right)
        elif node.data == 'implication':
            return (not interpret(left)) or interpret(right)
        elif node.data == 'intersection':
            return interpret(left) & interpret(right)
        elif node.data == 'union':
            return interpret(left) | interpret(right)
        elif node.data == 'difference':
            return interpret(left) - interpret(right)
        elif node.data == 'symmetric_difference':
            return interpret(left) ^ interpret(right)
        elif node.data == 'multiplication':
            return interpret(left) * interpret(right)
        else:
            raise Exception(f"ERROR! {node.data}")
    elif isinstance(node, Token):
        node = constant(node.value)
        return node
    else:
        raise Exception(f"ERROR! unkonown instance {type(node)}")

In [5]:
for p in tree.iter_subtrees_topdown():
    print(p)

Tree('implication', [Tree('disjunction', [Tree('lower_equal', [Tree('intersection', [Token('SET', 'A'), Token('SET', 'B')]), Tree('union', [Token('SET', 'A'), Token('SET', 'B')])]), Tree('greater_than', [Tree('multiplication', [Token('SET', 'A'), Token('SET', 'B')]), Tree('multiplication', [Token('SET', 'A'), Token('SET', 'B')])])]), Tree('implication', [Tree('conjunction', [Tree('not_equal', [Token('SET', 'A'), Token('SET', 'B')]), Tree('equal', [Token('SET', 'B'), Token('SET', 'B')])]), Tree('greater_equal', [Token('SET', 'A'), Tree('difference', [Token('SET', 'B'), Tree('symmetric_difference', [Token('SET', 'A'), Token('SET', 'B')])])])])])
Tree('disjunction', [Tree('lower_equal', [Tree('intersection', [Token('SET', 'A'), Token('SET', 'B')]), Tree('union', [Token('SET', 'A'), Token('SET', 'B')])]), Tree('greater_than', [Tree('multiplication', [Token('SET', 'A'), Token('SET', 'B')]), Tree('multiplication', [Token('SET', 'A'), Token('SET', 'B')])])])
Tree('lower_equal', [Tree('interse

In [6]:
print(interpret(tree))

True


In [7]:
A = {1,2,3}
B = {4,5,6}
C = {7,8,9}

R = BinaryRelation({(1,4),(1,5),(2,4),(3,6)},A,B)
S = BinaryRelation({(4,9),(5,8),(6,7)},B,C)

tree1 = parser.parse("R & S")  
tree2 = parser.parse("R|S")  
tree3 = parser.parse("R == R and S != R")  

print(interpret(tree1))
print(interpret(tree2))
print(interpret(tree3))

{}
{(1, 4),(1, 5),(2, 4),(3, 6),(4, 9),(5, 8),(6, 7)}
True
